# 텍스트 유사도

- 챗봇 엔진에 입력되는 문장과 시스템에서 해당 주제의 답변과 연관되어 있는 질문이 얼마나 유사한지 계산할 수 있어야 적절한 답변을 출력할 수 있음
- 두 문장 간의 유사도를 계산하기 위해서는 문장 내에 존재하는 단어들을 수치화해야 함
    - 이 때 언어 모델에 따라 통계를 이용하는 방법과 인공 신경망을 이용하는 방법으로 나눌 수 있음
    - word2vec은 인공 신경망 방식
    - 인공 신경망 방식이 절대적으로 좋은 것은 아니며 상황에 따라 통계적인 방식이 더 적절할 수 있음

## n-gram 유사도

- 주어진 문장에서 n개의 연속적인 단어 시퀀스(단어 나열)을 의미
- n개의 단어를 토큰으로 사용
- 이웃한 단어의 출현 횟수를 통계적으로 표현해 텍스트의 유사도를 계산하는 방법

- n-gram 예시

- 문장 : 1661년 6월 뉴턴은 선생님의 제안으로 트리니티에 입학하였다.
    - n = 1 : 1661년 / 6월 / 뉴턴 / 선생님 / 제안 / 트리니티 / 입학
    - n = 2 : 1661년 6월 / 6월 뉴턴 / 뉴턴 선생님 / 선생님 제안 / 제안 트리니티 / 트리니티 입학
    - n = 3 : 1661년 6월 뉴턴 / 6월 뉴턴 선생님 / 뉴턴 선생님 제안 / 선생님 제안 트리니티 / 제안 트리니티 입학

### n-gram 을 이용한 문장 간 유사도 계싼

- 문장을 n-gram 으로 토큰을 분리한 후 단어 문서 행렬을 만든 후 두 문장을 서로 비교해 동일한 단어의 출현 빈도를 확률로 계산해 유사도를 구함

- 수식 : 두 문장 A와 B에서 동일한 토큰의 출현 빈도 / A 문장의 전체 토큰 수
    - 기준이 되는 문장 A에서 나온 전체 토큰 중에서 A와 B에 동일한 토큰이 얼마나 있는지 비율로 표현
    - 1에 가까울수록 B가 A에 유사함

In [1]:
from konlpy.tag import Komoran

In [2]:
# 어절 단위 n-gram
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text)) if len(text[x:]) >= num_gram]
    return tuple(ngrams)

In [3]:
# 음절 n-gram 분석
def phoneme_ngram(bow, num_gram):
    sentence = " ".join(bow)
    text = tuple(sentence)
    slen = len(text)
    ngrams = [text[x:x + num_gram] for x in range(0, slen) if len(text[x:]) >= num_gram]
    return tuple(ngrams)

In [4]:
# 유사도 계산
def similarity(doc1, doc2):
    cnt = 0
    for token in doc1:
        if token in doc2:
            cnt = cnt + 1
            
    return cnt / len(doc1)

- doc1의 토큰이 doc2의 토큰과 얼마나 동일한지 횟수를 카운트
- 카운트 된 값을 doc1의 전체 토큰 수로 나누면 유사도가 계산됨

In [5]:
# 문장 정의
sentence1 = "6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다"
sentence2 = "6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다"
sentence3 = "나는 맛있는 밥을 뉴턴 선생님과 함께 먹었습니다"

In [7]:
# 형태소 분석기에서 명사 추출
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

In [8]:
# 추출된 명사 토큰 출력
print(bow1)
print(bow2)
print(bow3)

['6월', '뉴턴', '선생님', '제안', '트리니티', '입학']
['6월', '뉴턴', '선생님', '제안', '대학교', '입학']
['밥', '뉴턴', '선생', '님과 함께']


In [9]:
# 단어 n-gram 토큰 추출
doc1 = word_ngram(bow1, 2)
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

In [10]:
# 추출된 n-gram 토큰 출력
print(doc1)
print(doc2)
print(doc3)

(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'))
(('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'))


In [11]:
# 유사도 계산
r1 = similarity(doc1, doc2)
r2 = similarity(doc3, doc1)
print(r1)
print(r2)

0.6
0.0


- n-gram은 문장에 존재하는 모든 단어의 출현 빈도를 확인하는 것이 아니라 연속되는 문장에서 일부 단어만 확인
- 전체 문장을 고려한 언어 모델보다 정확도가 떨어질 수 있음
- n을 크게 잡을수록 비교 문장의 토큰과 비교할 때 카운트를 놓칠 확률이 커짐
- n을 작게 잡을수록 카운트 확률은 높아지지만 문맥을 파악하는 정확도는 떨어질 수 밖에 없음
- 일반적으로 n은 1 ~ 5 사이의 값을 사용